![Data Engineering](img/header2.JPG "Data Engineering")

# <span style="color: darkblue">Voter Turnout in Presidential Elections</span>

We will use this notebook and project to explore, visualize, and analyze presidential election participation data, otherwise known as "voter turnout". Elections are voluntary in the United States and understanding the spatial characteristics of voter turnout is very important to understand how policy decisions are made based on who participated in the election process. 

We will begin with <span style="color:purple">Data Engineering</span>: Broadly described as the actions taken to make data useful for analysis. Whether it’s removing records with erroneous data, reformatting the structure of a table to better conduct analysis, or several other actions that may help you prepare data, Data Engineering is an important part of every analysis. 

We will start by downloading and preparing US presidential election [data from MIT's Election Data and Science Lab](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/VOQCHQ), including handling missing values, reformatting data types, and restructuring the format of a table. Using GIS, Data Engineering can then make use of geocoding and geoenrichment to further prepare our data for analysis.

***

## Step 1: Load and Clean Election Data

Goals:
- Handle missing values
- Correct truncated zeroes with FIPS field
- Restructure table format

To download and prepare the election data available, we will use a mix of Pandas, ArcPy, and the ArcGIS API for Python. Dataframes in Pandas serve as an effective way to format data and fix issues. First, let's import the necessary modules.

##### Import needed modules

In [ ]:
# The import statements load each module
import arcgis
import pandas as pd
import os
import arcpy  # Best practice: Load arcpy last to maintain priority for namespace

##### Read data into Python

In [ ]:
# Make reference to the file path for the csv, which should be in the same directory as the notebook
table_csv_path = "countypres_2000-2016.csv"

# Use Pandas to read the csv into a dataframe
data_df = pd.read_csv(table_csv_path, dtype={'year': str, 'FIPS': str})  # dtype parameter specifies that year and FIPS fields are string

# Use the head function to display the first five records of the dataframe
data_df.head()

### Handle missing data 

![Null Values](img/null_values.gif "Null Values")

In [ ]:
# Set the field to check nulls for
field_to_check = "FIPS"

# Determine how many rows are in the table
rowcount = data_df.shape[0]

# Determine how many rows have null FIPS 
null_fips_rowcount = data_df.loc[data_df[field_to_check].isnull()].shape[0]

# Calculate how much of the data this represents as a percentage
percentage_null_fips = round((null_fips_rowcount / rowcount) * 100, 2)

# Use a print statement to report this information
print("There were "+str(null_fips_rowcount)+" records with null "+str(field_to_check)+" values in the data.\nThis amounts to " +str(percentage_null_fips)+"% of the available data.")

In [ ]:
# Use the notnull function and the loc function to create a new dataframe without null FIPS records
data_df = data_df.loc[data_df['FIPS'].notnull()]

### Explore and handle data types

![fix_truncated_zeroes](img/trunc_zeroes.gif "Fix Truncated Zeroes")

In [ ]:
# Get the first five records of the table
data_df.head()

In [ ]:
# Check how many records have a FIPS value with four characters
trunc_df = data_df.loc[data_df['FIPS'].str.len() == 4]
trunc_data_per = (trunc_df.shape[0] / data_df.shape[0])*100

# Use another print statement (using the f format key) to report this information
print(f"{round(trunc_data_per, 2)}% of data ({trunc_df.shape[0]} rows) has truncated FIPS values.")

Next, we need to fix the FIPS field since the data has leading zeroes truncated by its interpretation as a numeric field. We can create a simple function in python to determine if the value is four characters, and append a leading zero if that's the case. 

In [ ]:
# Define a helper function to fix truncated zeros, with one parameter: the value to be processed
def fix_trunc_zeros(val):
    # Use an if statement to check if there are four characters in the string representation of the value
    if len(str(val)) == 4:
        # If this is the case, return the value with an appended "0" in the front
        return "0"+str(val)
    # Otherwise...
    else:
        # Return the value itself
        return str(val)

In [ ]:
# Test helper function with truncated value
fix_trunc_zeros(7042)  # You should see an appended zero: "07042"

In [ ]:
# Run helper function on the FIPS field using the apply and lambda method 
data_df['FIPS'] = data_df['FIPS'].apply(lambda x: fix_trunc_zeros(x))

# Print information on the operation performed, and show the first few records to confirm it worked
print(f"{round(trunc_data_per, 2)}% of data ({trunc_df.shape[0]} rows) had truncated FIPS IDs corrected.")
data_df.head()

### Reformat the table structure

![reformat_table](img/reformat_table.gif "Reformat Table")

We now need to reformat the structure of the table. Currently, each record corresponds to a candidate and their votes in a particular county. We need each record to correspond to each county, with fields showing the votes for different candidates for that election year. 

The animation above displays the reformatting necessary. It is possible to do this using Excel pivot tables, but a Python script to perform this might make things a bit easier to automate and share. The following code cell performs all the actions displayed above in the animation: 

In [ ]:
# Reformat the dataframe by setting a multiindex (set_index with multiple fields) and pivoting the table (unstack)
df_out = data_df.set_index(['FIPS', 
                            'year', 
                            'county', 
                            'state', 
                            'state_po', 
                            'office', 
                            data_df.groupby(['FIPS', 'year']).cumcount()+1]).unstack()

# Use the indexes for the columns to set column names (Ex: candidate_1, candidate_2, votes_1, votes_2, etc.)
df_out.columns = df_out.columns.map('{0[0]}_{0[1]}'.format)

# Rename columns 
df_out = df_out.rename(columns={"candidate_1": "candidate_dem",
                                "candidatevotes_1": "votes_dem",
                                "candidate_2": "candidate_gop",
                                "candidatevotes_2": "votes_gop",
                                "totalvotes_1": "votes_total",
                                "state_po": "state_abbrev"
                                })

# Keep only the necessary columns
df_out = df_out[["candidate_dem", "votes_dem",
                 "candidate_gop", "votes_gop",
                 "votes_total"]]

# Remove the multiindex since we no longer need these fields to be "locked" for the pivot
df_out.reset_index(inplace=True)

# Print out the first few records to confirm everything worked
df_out.head()

**Let's break this down step by step and fully understand it.**

First, take a few seconds to rewatch the animation above again and consider each step. 

In essence, we need to:

1. Set a few fields aside, "locking" them from the table pivot. 
2. Pivot the table using the remaining fields.
3. Give the pivoted fields designations for each party. 
4. Bring the locked fields back to our table. 

For context, Pandas has the following powerful capabilities that help us perform this operation: 

- The ability to set an index using multiple fields, which acts as our "locking" mechanism for step 1. 
- The ability to perform an operation using a "groupby" function, which lets us group each observation by candidate/party
- The ability to unstack a table, which lets us handle the table pivot.

**1. Setting an index using multiple fields** allows us to designate specific fields as index fields, which will not be impacted when a table pivot occurs. In other words, the fields specified in the set_index part of code above are "locked" when the later part of the code performs the table pivot via the "unstack" function.  

In [ ]:
# Notice how running this cell uses the specified fields as row indices, 
# which prevents them from being "rotated" in the table pivot
data_df.set_index(['FIPS', 
                   'year', 
                   'county', 
                   'state', 
                   'state_po', 
                   'office'])

**2. The built-in groupby function** allows us to perform an operation using the unique values from a specified set of fields. This is useful because we can then count how many rows exist for a given FIPS and Year combination, which essentially lets us group data by the candidate that it pertains to. See the following example: 

In [ ]:
data_df.set_index(['FIPS', 
                   'year', 
                   'county', 
                   'state', 
                   'state_po', 
                   'office', 
                   data_df.groupby(['FIPS', 'year']).cumcount()+1])

**3. The ability to unstack a table** allows us to perform the table pivot, which essentially "rotates" the table and makes rows into columns (or columns into rows).

In [ ]:
data_df.set_index(['FIPS', 
                   'year', 
                   'county', 
                   'state', 
                   'state_po', 
                   'office', 
                   data_df.groupby(['FIPS', 'year']).cumcount()+1]).unstack()

**4. Finally, we put it all together and rename the output columns**, using the pandas ability to rename fields and removing extraneous fields we no longer need. This produces the table format we need.

In [ ]:
# Reformat the dataframe by setting a multiindex (set_index with multiple fields) and pivoting the table (unstack)
df_out = data_df.set_index(['FIPS', 
                            'year', 
                            'county', 
                            'state', 
                            'state_po', 
                            'office', 
                            data_df.groupby(['FIPS', 'year']).cumcount()+1]).unstack()

# Use the indexes for the columns to set column names (Ex: candidate_1, candidate_2, votes_1, votes_2, etc.)
df_out.columns = df_out.columns.map('{0[0]}_{0[1]}'.format)

# Rename columns 
df_out = df_out.rename(columns={"candidate_1": "candidate_dem",
                                "candidatevotes_1": "votes_dem",
                                "candidate_2": "candidate_gop",
                                "candidatevotes_2": "votes_gop",
                                "totalvotes_1": "votes_total",
                                "state_po": "state_abbrev"
                                })

# Keep only the necessary columns
df_out = df_out[["candidate_dem", "votes_dem",
                 "candidate_gop", "votes_gop",
                 "votes_total"]]

# Remove the multiindex since we no longer need these fields to be "locked" for the pivot
df_out.reset_index(inplace=True)

# Print out the first few records to confirm everything worked
df_out.head()

### Calculate Additional Columns/Features

##### Calculate total votes for non major party

In [ ]:
# Calculate votes that did not go for the Democrat or Republican party
df_out['votes_other'] = df_out['votes_total'] - (df_out['votes_dem'] + df_out['votes_gop'])
df_out.head()

##### Create additional attributes (voter percentages and raw differences)

In [ ]:
# Calculate voter share attributes
df_out['voter_share_major_party'] = (df_out['votes_dem'] + df_out['votes_gop']) / df_out['votes_total']
df_out['voter_share_dem'] = df_out['votes_dem'] / df_out['votes_total']
df_out['voter_share_gop'] = df_out['votes_gop'] / df_out['votes_total']
df_out['voter_share_other'] = df_out['votes_other'] / df_out['votes_total']

# Calculate raw difference attributes
df_out['rawdiff_dem_vs_gop'] = df_out['votes_dem'] - df_out['votes_gop']
df_out['rawdiff_gop_vs_dem'] = df_out['votes_gop'] - df_out['votes_dem']
df_out['rawdiff_dem_vs_other'] = df_out['votes_dem'] - df_out['votes_other']
df_out['rawdiff_gop_vs_other'] = df_out['votes_gop'] - df_out['votes_other']
df_out['rawdiff_other_vs_dem'] = df_out['votes_other'] - df_out['votes_dem']
df_out['rawdiff_other_vs_gop'] = df_out['votes_other'] - df_out['votes_gop']

# Calculate percent difference attributes
df_out['pctdiff_dem_vs_gop'] = (df_out['votes_dem'] - df_out['votes_gop']) / df_out['votes_total']
df_out['pctdiff_gop_vs_dem'] = (df_out['votes_gop'] - df_out['votes_dem']) / df_out['votes_total']
df_out['pctdiff_dem_vs_other'] = (df_out['votes_dem'] - df_out['votes_other']) / df_out['votes_total']
df_out['pctdiff_gop_vs_other'] = (df_out['votes_gop'] - df_out['votes_other']) / df_out['votes_total']
df_out['pctdiff_other_vs_dem'] = (df_out['votes_other'] - df_out['votes_dem']) / df_out['votes_total']
df_out['pctdiff_other_vs_gop'] = (df_out['votes_other'] - df_out['votes_gop']) / df_out['votes_total']

df_out.head()

***

## Step 2: Geoenable election data

Goals:
   - Retrieve 2016 USA county population GIS data
   - Perform join, bringing geometry to election data
   - Calculate Voter Turnout per county
   - Convert dataframe to feature class

We will now bring geometry data for each county into the table. 

### Retrieve 2016 USA county population GIS data

A useful source of data is the [ArcGIS Living Atlas of the World](https://livingatlas.arcgis.com), where we can find a service containing [Voting Age Population totals (citizens aged 18+) for each county](https://www.arcgis.com/home/item.html?id=2e8aaf91178c4c91b974d0bc4234dbfa). 

This dataset is included with this project to allow you to append the election data to county geometry and allow us to calculate voting turnout for each county. The next few cells will load this feature class into a spatially-enabled dataframe and allow us to geoenable the election data we have been preparing. Additionally, this feature class allows us to actually calculate voter turnout since it includes voting-age population counts for 2016. 

We will now reference this ArcGIS Pro project and its file geodatabase using ArcPy as well. Let's get started.

In [ ]:
# Authenticate with a GIS using the ArcGIS API for Python
gis = arcgis.gis.GIS()

In [ ]:
# Search for USA_Counties
search = gis.content.search("USA Counties", item_type="feature_service", outside_org=True, sort_field="numViews")
# Use the correct index to reference the search result
counties_item = search[2]
counties_item

In [ ]:
# Read the layer into a dataframe
counties_df = pd.DataFrame.spatial.from_layer(counties_item.layers[0])
counties_df = counties_df[['FIPS', 'NAME', 'STATE_FIPS', 'STATE_NAME', 'OBJECTID', 'POPULATION', 'POP_SQMI', 'SHAPE', 'Shape_Area', 'Shape_Leng', 'Shape__Area', 'Shape__Length']]
counties_df.head()

### Perform join, bringing geometry and population column to election data

We now have a dataframe with election data ('df_out') and a spatially-enabled dataframe of county voting-age population data ('counties_df'). Let's merge the datasets. 

In [ ]:
# Join the data to our election data table
geo_df = pd.merge(df_out, counties_df, left_on='FIPS', right_on="FIPS", how='left')
# Visualize the merged data, notice the SHAPE column at the end
geo_df.head()

In [ ]:
# Create a copy of the data, and perform a query
data_2016_df = geo_df.copy()
data_2016_df.query("year == '2016'", inplace=True)
data_2016_df.head()

### Convert dataframe to feature class

We can now finally convert our data to feature classes! The ArcGIS API for Python (which was invoked by using "import arcgis") lets us export the spatially-enabled dataframe to a feature class so we can do further analysis.

**Note: Executing the following cell may take a few minutes**

In [ ]:
# Create a subset of the data just for the 2016 presidential election
fgdb = r"C:\Users\albe9057\Documents\ArcGIS\Projects\Data Engineering and Visualization\Data Engineering and Visualization.gdb"
out_2016_fc_name = "county_elections_pres_2016"
out_2016_fc = data_2016_df.spatial.to_featureclass(os.path.join(fgdb, out_2016_fc_name))
out_2016_fc

Select the Map tab (titled "Data Engineering"), and view the added layers to the Table of Contents. You will now see counties across the United States with 2016 voting turnout data.

***

## Step 3: Geoenrich election data

Goals:
   - Use Geoenrichment to bring demographic and socioeconomic variables to use in analysis

Geoenrichment in ArcGIS Pro allows us to add columns of data for each county that can help us analyze relationships and potentially model voter turnout. Geoenrichment can be performed using ArcPy, but we recommend that you use the GeoProcessing Enrich tool to explore potential variables that may help explain voter turnout. You may now reference the MOOC guidance for the steps covering how to geoenrich this data. 

***